# Dataset Overview
A quick look at each processed dataset used in the ICD coding pipeline.

In [1]:
import polars as pl
pl.Config.set_fmt_str_lengths(120)
pl.Config.set_tbl_rows(5)

polars.config.Config

---
## 1. MIMIC-III
Critical care database from Beth Israel Deaconess Medical Center (2001-2012). Contains discharge summaries linked to ICD-9 diagnosis and procedure codes. The `_full` variant has all admissions, `_clean` removes rare codes (<10 occurrences), and `_50` keeps only the top 50 Mullenbach codes.

In [2]:
mimiciii = pl.read_parquet("../data/mimic-iii/processed/mimiciii_full.parquet")
print(f"Shape: {mimiciii.shape}")
print(f"Columns: {mimiciii.columns}")
print(f"Unique patients: {mimiciii['subject_id'].n_unique()}")
print(f"Unique admissions: {mimiciii['hadm_id'].n_unique()}")
print(f"Avg codes per admission: {mimiciii['codes'].list.len().mean():.1f}")
print(f"Avg text length (chars): {mimiciii['text'].str.len_chars().mean():.0f}")

Shape: (52722, 11)
Columns: ['subject_id', 'hadm_id', 'note_type', 'text', 'note_subtype', 'note_id', 'diagnosis_code_type', 'diagnosis_codes', 'procedure_code_type', 'procedure_codes', 'codes']
Unique patients: 41126
Unique admissions: 52722
Avg codes per admission: 15.9
Avg text length (chars): 10883


In [3]:
row = mimiciii.row(0, named=True)
print(f"Subject: {row['subject_id']}, Admission: {row['hadm_id']}")
print(f"Codes ({len(row['codes'])}): {row['codes'][:10]}")
print(f"\nText preview ({len(row['text'])} chars):\n{row['text'][:500]}...")

Subject: 8387, Admission: 186408
Codes (12): ['99.83', '765.19', '770.81', 'V30.01', '753.29', 'V05.3', '774.2', '779.81', '765.28', '766.1']

Text preview (7134 chars):
Admission Date: [**2168-12-11**]        Discharge Date:[**2169-1-3**]

Date of Birth:  [**2168-12-11**]        Sex:  M

Service:  NB


HISTORY OF PRESENT ILLNESS:  Baby [**Name (NI) **] [**Known lastname 1071**] was admitted to
the Neonatal Intensive Care Unit at 4 days of life from the
Newborn Nursery for evaluation of dusky episodes.

The infant is a 3285 gram product of a 36 and 1/7 weeks
gestation pregnancy born to a 40 year old, G2, para 1, now 2
mother.  Prenatal laboratory studies include...


---
## 2. MIMIC-IV
Updated critical care database (2008-2019) with ICD-10 codes. Larger than MIMIC-III and uses the more granular ICD-10-CM/PCS coding system.

In [4]:
mimiciv = pl.read_parquet("../data/mimic-iv/processed/mimiciv.parquet")
print(f"Shape: {mimiciv.shape}")
print(f"Columns: {mimiciv.columns}")
print(f"Unique patients: {mimiciv['subject_id'].n_unique()}")
print(f"Unique admissions: {mimiciv['hadm_id'].n_unique()}")
print(f"Avg text length (chars): {mimiciv['text'].str.len_chars().mean():.0f}")

Shape: (331677, 12)
Columns: ['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq', 'charttime', 'storetime', 'text', 'diagnosis_code_type', 'diagnosis_codes', 'procedure_code_type', 'procedure_codes']
Unique patients: 145855
Unique admissions: 331668
Avg text length (chars): 10552


In [5]:
row = mimiciv.row(0, named=True)
print(f"Subject: {row['subject_id']}, Admission: {row['hadm_id']}")
print(f"Diagnosis codes: {row['diagnosis_codes']}")
print(f"Procedure codes: {row['procedure_codes']}")
print(f"\nText preview ({len(row['text'])} chars):\n{row['text'][:500]}...")

Subject: 17409226, Admission: 25292688
Diagnosis codes: ['E78.5', 'Y92.009', 'E87.1', 'G40.909', 'I50.33', 'T42.75XA', 'D50.9', 'H40.10X0', 'J43.9', 'D51.9', 'J45.909', 'I69.320', 'J69.0', 'Z86.718', 'J84.9', 'I69.351', 'T42.1X5A', 'F41.9', 'E03.9', 'M47.892', 'T50.1X5A', 'E87.8', 'I10', 'D63.8', 'Z79.01']
Procedure codes: None

Text preview (22359 chars):
 
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: NEUROLOGY
 
Allergies: 
NSAIDS (Non-Steroidal Anti-Inflammatory Drug) / aspirin
 
Attending: ___.
 
Chief Complaint:
AMS
 
Major Surgical or Invasive Procedure:
None
 
History of Present Illness:
Ms. ___ is an ___ year old woman with a past medical history 
of seizure disorder (manifested by L arm/face parasthesias), L 
MCA stroke in setting of...


---
## 3. MedDec
Span-level annotations on MIMIC-III discharge summaries marking medical decisions (treatments, tests, etc.) with categories like `medication`, `therapeutic`, `diagnostic`. Each annotation has start/end offsets into the clinical text.

In [6]:
meddec = pl.read_parquet("../data/meddec/processed/meddec.parquet")
print(f"Shape: {meddec.shape}")
print(f"Columns: {meddec.columns}")
print(f"Unique notes: {meddec['hadm_id'].n_unique()}")
print(f"Categories: {meddec['category'].unique().to_list()}")
print(f"Decisions: {meddec['decision'].unique().to_list()}")
print(f"Annotations per category:")
print(meddec.group_by("category").len().sort("len", descending=True))

Shape: (50047, 12)
Columns: ['annotator_id', 'start_offset', 'end_offset', 'category', 'decision', 'annotation_id', 'file_name', 'subject_id', 'hadm_id', 'note_id', 'note_id_right', 'text']
Unique notes: 399
Categories: ['Category 11: Other - (no result)', 'Category 11: Other - Therapeutic procedure attempted but failed', 'Category 9: Advice and precaution', 'Category 8: Deferment', 'Category 11: Other - Further laboratory work-up recommended', 'Category 6: Therapeutic procedure related', 'Category 7: Evaluating test result', 'Category 11: Other - Monitoring labs', 'Category 11: Other - No mention to what service the patient was transferred to', 'Category 11: Other - Patient decision to go home without medical advice', 'Category 11: Other - Further monitoring needed', 'Category 2: Gathering additional information', 'Category 5: Drug related', 'Category 11: Other - Pending diagnostic test', 'Category 4: Treatment goal', 'Category 11: Other - Medical devices/materials', 'TBD', 'Category 

In [7]:
row = meddec.filter(pl.col("text").is_not_null()).row(0, named=True)
print(f"Admission: {row['hadm_id']}, Category: {row['category']}, Decision: {row['decision']}")
print(f"Span: [{row['start_offset']}:{row['end_offset']}]")
if row['text']:
    snippet = row['text'][max(0,row['start_offset']-50):row['end_offset']+50]
    print(f"\nContext around annotation:\n...{snippet}...")

Admission: 115941, Category: Category 3: Defining problem, Decision: new brain mass
Span: [294:308]

Context around annotation:
...Name3 (LF) 1854**]
Chief Complaint:
consulted for new brain mass

Major Surgical or Invasive Procedure:
[**5-26**]...


---
## 4. SNOMED Entity Linking
Annotations linking spans in MIMIC-IV clinical notes to SNOMED-CT concepts. Each row maps a text span (start/end offsets) to a SNOMED concept ID and name.

In [8]:
snomed = pl.read_parquet("../data/snomed/processed/snomed.parquet")
print(f"Shape: {snomed.shape}")
print(f"Columns: {snomed.columns}")
print(f"Unique notes: {snomed['note_id'].n_unique()}")
print(f"Unique concepts: {snomed['concept_id'].n_unique()}")
print(f"Top 10 concepts:")
print(snomed.group_by("concept_name").len().sort("len", descending=True).head(10))

Shape: (51574, 6)
Columns: ['note_id', 'start', 'end', 'concept_id', 'text', 'concept_name']
Unique notes: 204
Unique concepts: 5336
Top 10 concepts:
shape: (10, 2)
┌───────────────────────────────────────────┬─────┐
│ concept_name                              ┆ len │
│ ---                                       ┆ --- │
│ str                                       ┆ u32 │
╞═══════════════════════════════════════════╪═════╡
│ No abnormality detected                   ┆ 584 │
│ Red cell distribution width determination ┆ 489 │
│ Physical examination                      ┆ 455 │
│ …                                         ┆ …   │
│ Creatinine measurement                    ┆ 365 │
│ Fever                                     ┆ 359 │
└───────────────────────────────────────────┴─────┘


In [9]:
row = snomed.row(0, named=True)
print(f"Note: {row['note_id']}, Concept: {row['concept_name']} ({row['concept_id']})")
print(f"Span: [{row['start']}:{row['end']}]")
print(f"\nText preview ({len(row['text'])} chars):\n{row['text'][:500]}...")

Note: 10060142-DS-9, Concept: Allergy to penicillin (91936005)
Span: [179:190]

Text preview (4279 chars):
 
Name:  ___                  Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: SURGERY
 
Allergies: 
Penicillins
 
Attending: ___.
 
Chief Complaint:
Biliary pancreatitis
 
Major Surgical or Invasive Procedure:
___: Laparoscopic cholecystectomy

 
History of Present Illness:
Mr. ___ is a ___ man who had severe biliary 
pancreatitis resulting in pancreatic necrosis for which he was 
treated with nasojejunal feedings and...


---
## 5. MDACE

In [ ]:
# TODO to be downloaded and processed